<a href="https://colab.research.google.com/github/lehenbauer/colab/blob/main/JuicyJukebox_w_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#JuicyJukebox - The Fastest, Simplest, Jukebox Experience
Powered by Speed Upsampling by MichaelsLab


Last updated: 2022-09-30 at 09:45 AM EST

In [ ]:
#@title Verify Graphics Card
!nvidia-smi -L
your_lyrics = """
"""

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-bf59a4fe-778c-60e9-7f8b-04e6b9c24689)


In [ ]:
#Optional, paste lyrics below between the top and bottom groups of quotation marks.
your_lyrics = """
"""

In [ ]:
#@title Select your settings and run click "Restart and Run All"

gpu_info = !nvidia-smi -L
if gpu_info[0].find('Tesla T4') >= 0:
  model = "5b_lyrics"
  max_batch_size = 2
  print('Tesla T4 detected, model set to 5b_lyrics.')
elif gpu_info[0].find('Tesla K80') >= 0:
  model = "1b_lyrics"
  max_batch_size = 8
  print('Tesla K80 detected, model set to 1b_lyrics.')
elif gpu_info[0].find('Tesla P100') >= 0:
  model = "5b_lyrics"
  max_batch_size = 3
  print('Tesla P100 detected, model set to 5b_lyrics.')
elif gpu_info[0].find('Tesla V100') >= 0:
  model = "5b_lyrics"
  max_batch_size = 3
  print('Tesla V100 detected, model set to 5b_lyrics.')
elif gpu_info[0].find('A100') >= 0:
  model = "5b_lyrics"
  max_batch_size = 6
  print('A100 detected! Congrats!!! model set to 5b_lyrics.')
else:
  model = "5b_lyrics"
  max_batch_size = 3
  print('Different GPU detected, model set to 5b_lyrics.')

from google.colab import drive
drive.mount('/content/gdrive')

!pip install --upgrade git+https://github.com/craftmine1000/jukebox-saveopt.git

!cd /content/ ; wget https://d1a0pat3ioj7h1.cloudfront.net/system/jukebox/sample.py
!mv /content/sample.py /usr/local/lib/python3.7/dist-packages/jukebox/sample.py

import jukebox
import torch as t
import librosa
import os
from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
# MPI Connect. MPI doesn't like being initialized twice, hence the following
try:
    if device is not None:
        pass
except NameError:
    rank, local_rank, device = setup_dist_from_mpi()

max_split_size_mb = 16000


#START BPM CALCULATORS
import math

def hop_fract_correct(fraction,lsamples):
  return str(int((math.ceil((fraction*8192)//lsamples)*lsamples))/8192)

def hop_fract_correct_1b(fraction,lsamples):
  return str(int((math.ceil((fraction*6144)//lsamples)*lsamples))/6144)

def calcrenderduration(pduration,rduration):
  return (pduration+rduration)

def calclevelconditionbpm(setbpm,nbeats,tokensamples):
  return abs(((round((round((round(((((60/setbpm)*nbeats)*44100)/tokensamples),0)/8192),3)*8192),0)+1)/8192)-1)

def calclevelrenderbpm(setbpm,nbeats,tokensamples):
  return abs(((round((round((round(((((60/setbpm)*nbeats)*44100)/tokensamples),0)/8192),3)*8192),0)+1)/8192))

def calclevelrenderbpm_1b(setbpm,nbeats,tokensamples):
  return abs(((round((round((round(((((60/setbpm)*nbeats)*44100)/tokensamples),0)/6144),3)*6144),0)+1)/6144))

#END BPM CALCULATORS

save_and_load_models_from_drive = False #@param {type: "boolean"}
#@markdown Save/Load from GDrive loads models up to 8x faster, but takes up to 28GB in Drive storage.

#@markdown ---

#@markdown Recommended to leave artist/genre on "No Preference" if you are new to Jukebox.

select_artist = "No Preference"#@param  ["No Preference", "112", "*nsync", "10 years", "10cc", "16 volt", "2 chainz", "2 live crew", "2 unlimited", "2pac", "36 crazyfists", "38 special", "4him", "5 seconds of summer", "50 cent", "7 seconds", "a day to remember", "a tribe called quest", "aaron carter", "aaron copland", "aaron neville", "aaron tippin", "aaron watson", "abba", "abbey lincoln", "abc", "abney park", "acappella", "accept", "ace frehley", "ace of base", "acid drinkers", "adam ant", "adam green", "adam lambert", "adam sandler", "adolf busch", "adrian belew", "adriano celentano", "aerosmith", "aesop rock", "after all", "agathocles", "agnostic front", "agonoize", "agoraphobic nosebleed", "aiden", "aimee mann", "air", "air supply", "akon", "al green", "al jarreau", "al martino", "al stewart", "alabama", "alan jackson", "alan stivell", "albert collins", "albert king", "alborosie", "alejandro escovedo", "alesana", "alessandro scarlatti", "alexz johnson", "alfred cortot", "ali project", "alice cooper", "alice in chains", "alicia de larrocha", "alicia keys", "alison krauss", "alison moyet", "alkaline trio", "all", "all saints", "all that remains", "allison moorer", "alpha blondy", "alphaville", "altan", "alvin lee", "america", "amon amarth", "amorphis", "amos lee", "amy grant", "anastacia", "anathema", "and one", "andr previn", "andr s schiff", "andrea bocelli", "andrew lloyd webber", "andrew peterson", "andromeda", "andy partridge", "ane brun", "angel", "angelo branduardi", "angie stone", "angra", "animal collective", "anna ternheim", "anne clark", "anne murray", "anne sophie mutter", "anner bylsma", "annie lennox", "annihilator", "anthony hamilton", "anthony phillips", "anthrax", "anton n dvo k", "antonio meneses", "antonio vivaldi", "antony and the johnsons", "anvil", "aphex twin", "apocalyptica", "apologetix", "april wine", "arabesque", "arcade fire", "arcana", "arcangelo corelli", "architects", "archive", "arena", "aretha franklin", "ariana grande", "ariel pink", "arlo guthrie", "armand van helden", "armin van buuren", "armored saint", "arno", "art garfunkel", "arthur grumiaux", "arthur rubinstein", "artie shaw", "artillery", "artrosis", "artur schnabel", "arturo benedetti michelangeli", "as i lay dying", "ashanti", "asia", "asleep at the wheel", "asp", "asphyx", "assemblage 23", "astrud gilberto", "aswad", "atb", "atmosphere", "atomic kitten", "atreyu", "atrocity", "attila", "audio adrenaline", "august burns red", "autopsy", "avantasia", "avatar", "avenged sevenfold", "aviators", "avicii", "avulsed", "axel rudi pell", "axxis", "ayreon", "az", "b la bart k", "babbie mason", "baby bash", "babyface", "babyshambles", "backstreet boys", "backyard babies", "bad boys blue", "bad company", "bad religion", "badfinger", "badly drawn boy", "bananarama", "barbara mandrell", "barbra streisand", "barclay james harvest", "barry louis polisar", "barry white", "basement jaxx", "basshunter", "bathory", "bauhaus", "bay city rollers", "beach house", "beardfish", "beastie boys", "beat crusaders", "beau", "beaux arts trio", "bebo norman", "beck", "bed ich smetana", "bee gees", "behemoth", "belinda carlisle", "bell x1", "bella morte", "belle and sebastian", "belphegor", "ben lee", "benjamin britten", "benny benassi", "benny goodman", "beres hammond", "berlin", "bert jansch", "bessie smith", "beth hart", "beth nielsen chapman", "beth orton", "bethel music", "bette midler", "between the buried and me", "big audio dynamite", "big big train", "big bill broonzy", "big d and the kids table", "big joe turner", "big sean", "big tymers", "bill anderson", "bill monroe", "bill nelson", "bill withers", "billie holiday", "billy bragg", "billy currington", "billy gilman", "billy idol", "billy joe royal", "billy joe shaver", "billy joel", "billy ocean", "billy preston", "billy ray cyrus", "billy squier", "bing crosby", "biohazard", "birth control", "bizzy bone", "black flag", "black label society", "black moth super rainbow", "black sabbath", "black stone cherry", "black uhuru", "black veil brides", "blackfoot", "blake shelton", "blaze bayley", "bleeding through", "blind blake", "blind boy fuller", "blind guardian", "blind lemon jefferson", "blind willie mctell", "blindside", "blitzkid", "bloc party", "blondie", "blood on the dance floor", "blue cheer", "blue october", "blue rodeo", "blue stahli", "blue system", "blues traveler", "blutengel", "bo diddley", "boa", "boards of canada", "bob dylan", "bob rivers", "bob seger", "bobbie gentry", "bobby brown", "bobby darin", "bobby goldsboro", "bobby o", "bobby v", "bobby vee", "bobby vinton", "bobby womack", "body count", "bodyjar", "bolt thrower", "bon jovi", "bonfire", "bonnie raitt", "bonnie tyler", "borknagar", "boston", "bow wow", "boy george", "boyce avenue", "boysetsfire", "boyz ii men", "boyzone", "boz scaggs", "bracket", "brad paisley", "brainstorm", "brandi carlile", "brandy", "brazzaville", "bread", "breaking benjamin", "brenda lee", "brentalfloss", "brian eno", "brian hyland", "brian mcknight", "brian setzer", "brian wilson", "bride", "bright eyes", "bring me the horizon", "british sea power", "britney spears", "brockhampton", "broken social scene", "brook benton", "brotha lynch hung", "brownie mcghee", "bruce cockburn", "bruce dickinson", "bruce hornsby", "bruce springsteen", "bruno mars", "bryan adams", "bryan ferry", "bt", "bts", "buck 65", "buck owens", "buckcherry", "buddy guy", "buddy holly", "budgie", "building 429", "bukka white", "bullet for my valentine", "bumblefoot", "bun b", "bunny wailer", "burl ives", "burning heads", "burt bacharach", "burton cummings", "busta rhymes", "buzzcocks", "cab calloway", "caetano veloso", "cage", "calexico", "caliban", "calvin harris", "camel", "can", "cancerslug", "candlemass", "canibus", "canned heat", "cannibal corpse", "capercaillie", "capleton", "captain beefheart and the magic band", "car seat headrest", "caravan", "carbon leaf", "carcass", "carly simon", "carman", "carmen mcrae", "carole king", "carpenters", "carrie newcomer", "carrie underwood", "carter the unstoppable sex machine", "cass mccombs", "cassandra wilson", "casting crowns", "cat power", "cat stevens", "cathedral", "catie curtis", "cave in", "cece winans", "cee lo green", "celldweller", "celph titled", "celtic thunder", "celtic woman", "centinex", "chaka khan", "chamillionaire", "charles aznavour", "charles harrison", "charley pride", "charlie landsborough", "charlie louvin", "charlie musselwhite", "charlie peacock", "charlie rich", "charlotte martin", "cheap trick", "chenoa", "cher", "cheryl cole", "chet atkins", "chevelle", "chic", "chicago", "childish gambino", "children of bodom", "china crisis", "chris brown", "chris cornell", "chris de burgh", "chris isaak", "chris knight", "chris ledoux", "chris rea", "chris tomlin", "chris young", "christian bautista", "christina aguilera", "christoph eschenbach", "chubby checker", "chuck berry", "ciara", "cilla black", "circa survive", "circle jerks", "city and colour", "clan of xymox", "clannad", "clara haskil", "claude debussy", "claudio arrau", "clawfinger", "clay aiken", "clay walker", "cliff richard", "clint black", "closterkeller", "club 8", "cocorosie", "cocteau twins", "coil", "colbie caillat", "cold chisel", "colin blunstone", "colin hay", "colin james", "collective soul", "collin raye", "color me badd", "combichrist", "common", "con funk shun", "connie francis", "connie smith", "conor oberst", "converge", "conway twitty", "coolio", "corb lund", "corey hart", "corrosion of conformity", "countess", "country joe mcdonald", "covenant", "cowboy junkies", "cows", "cracker", "cradle of filth", "craig david", "creed", "creedence clearwater revival", "cross canadian ragweed", "crowded house", "cruachan", "crystal gayle", "crystal kay", "crystal lewis", "culture", "culture club", "current 93", "cursive", "curtis mayfield", "cypress hill", "d12", "daft punk", "dagoba", "damian marley", "damien jurado", "dan fogelberg", "dan seals", "dana winner", "dance gavin dance", "daniel lanois", "daniel lavoie", "daniel m ller schott", "daniel shafran", "danielson", "danko jones", "danzig", "dar williams", "darius rucker", "dark moor", "dark tranquillity", "darkest hour", "darkseed", "darkthrone", "darlene zschech", "daryl hall", "dashboard confessional", "dave alvin", "dave clark five", "dave davies", "dave dudley", "dave edmunds", "david and the giants", "david cassidy", "david crosby", "david crowder band", "david essex", "david geringas", "david knopfler", "david lee roth", "david meece", "david oistrakh", "david rovics", "david usher", "david wilcox", "dc talk", "de la soul", "dead can dance", "dead kennedys", "deadmau5", "dean martin", "deana carter", "death angel", "death cab for cutie", "death grips", "death in june", "debbie gibson", "debby boone", "deep purple", "deer tick", "deerhunter", "def leppard", "deftones", "degarmo and key", "deicide", "deine lakaien", "del shannon", "delbert mcclinton", "deliverance", "delta goodrem", "demi lovato", "demis roussos", "demon", "demon hunter", "deniece williams", "dennis brown", "descendents", "desmond dekker", "destroyer", "destruction", "dethklok", "devendra banhart", "devildriver", "devin townsend", "devin townsend project", "devo", "diamond rio", "diana krall", "diana ross", "dianne reeves", "diary of dreams", "dick haymes", "diddy", "dido", "die happy", "die krupps", "die toten hosen", "dimmu borgir", "dinah washington", "dino ciani", "dinu lipatti", "dio", "dion", "dionne warwick", "diorama", "dir en grey", "dire straits", "dirty heads", "disciple", "disclosure", "disturbed", "dixie chicks", "dizzee rascal", "dj shadow", "dmitri shostakovich", "dmx", "doc watson", "dog fashion disco", "dogwood", "dokken", "dolly parton", "don francisco", "don gibson", "don mclean", "don williams", "donna summer", "donovan", "dope", "doris day", "doro", "dottie west", "dougie maclean", "dover", "dragonforce", "drake", "dream evil", "dream theater", "dreamtale", "dropkick murphys", "drowning pool", "duke ellington", "dusty springfield", "dwight yoakam", "dying fetus", "eagles", "echolyn", "ed bruce", "ed harcourt", "ed sheeran", "eddi reader", "eddie from ohio", "eddie money", "eddie rabbitt", "eddy arnold", "eddy grant", "edenbridge", "edge of sanity", "edguy", "editors", "edvard grieg", "edwin fischer", "eighteen visions", "ektomorf", "elbow", "electric light orchestra", "electric six", "electric wizard", "eleni mandell", "elephant man", "elevation worship", "elf power", "elisa", "ella fitzgerald", "ella mae morse", "ellie goulding", "elliott smith", "ellis paul", "elmore james", "eloy", "elton john", "eluveitie", "elvenking", "elvis costello", "elvis presley", "elysian fields", "emil gilels", "emilie autumn", "eminem", "emmerson nogueira", "emmure", "emmylou harris", "en vogue", "enigma", "enrico caruso", "enrique iglesias", "ensiferum", "enslaved", "entombed", "epica", "epmd", "erasure", "eric bibb", "eric carmen", "eric church", "eric clapton", "eric johnson", "erick sermon", "ernest tubb", "erykah badu", "escape the fate", "esham", "esther ofarim", "ethel waters", "etta james", "eugenio finardi", "europe", "eurythmics", "eva cassidy", "evanescence", "evergrey", "every time i die", "everything but the girl", "evgeny kissin", "example", "exciter", "exodus", "extreme", "face to face", "fairport convention", "faith evans", "faith hill", "faith no more", "faithless", "falco", "falconer", "fall out boy", "falling up", "faron young", "fat joe", "fatboy slim", "fates warning", "fats domino", "fats waller", "faz l say", "fear factory", "felix mendelssohn bartholdy", "ferlin husky", "fernando ortega", "ffh", "field music", "fifteen", "fifth harmony", "fightstar", "firewind", "fish", "five finger death punch", "five iron frenzy", "flame", "fleetwood mac", "flo rida", "florida georgia line", "flying lotus", "flyleaf", "foetus", "foghat", "foreigner", "four tops", "fr d ric chopin", "francis poulenc", "franco battiato", "frank sinatra", "frank turner", "frank zappa", "frankie j", "frankie valli", "frankjavcee", "franz ferdinand", "franz liszt", "franz schubert", "freak kitchen", "fred astaire", "fred eaglesmith", "freddie hart", "freddie king", "freedom call", "frenzal rhomb", "frightened rabbit", "fritz kreisler", "front 242", "front line assembly", "fu manchu", "fugazi", "fun people", "funeral for a friend", "funkadelic", "funker vogt", "furry lewis", "fury in the slaughterhouse", "future", "gabrielle", "gaelic storm", "gaetano donizetti", "gaither vocal band", "galactic cowboys", "gamma ray", "gang starr", "garrick ohlsson", "garth brooks", "gary allan", "gary glitter", "gary moore", "gary numan", "gary stewart", "gene autry", "gene clark", "gene pitney", "gene vincent", "gene watson", "genesis", "gentle giant", "gentleman", "geoff moore", "georg philipp telemann", "george benson", "george clinton", "george formby", "george frideric handel", "george gershwin", "george harrison", "george jones", "george michael", "george strait", "georges bizet", "georges cziffra", "gerald levert", "gerald moore", "gerry rafferty", "geto boys", "ghostface killah", "giacomo puccini", "gidon kremer", "gil shaham", "gilberto gil", "gino vannelli", "ginuwine", "gioacchino rossini", "girls aloud", "girlschool", "giuseppe verdi", "glee cast", "glen campbell", "glenn frey", "glenn gould", "glenn hughes", "glenn miller", "gloria estefan", "gloria gaynor", "god module", "godsmack", "golden earring", "goldfinger", "goldfrapp", "good riddance", "goodie mob", "gordon bok", "gordon lightfoot", "gorillaz", "gotthard", "grace jones", "gramatik", "grand funk railroad", "grandaddy", "grateful dead", "grave", "grave digger", "graveworm", "great big sea", "great white", "greeley estates", "greg brown", "gregg allman", "gregor piatigorsky", "gregorian", "gregory isaacs", "grigory sokolov", "grimskunk", "grinspoon", "grits", "groundation", "guano apes", "guardian", "guiomar novaes", "gustav mahler", "guy clark", "guy mitchell", "guy sebastian", "gwar", "gwen stefani", "gym class heroes", "haemorrhage", "halestorm", "half man half biscuit", "halford", "hammerfall", "hank locklin", "hank snow", "hank thompson", "hank williams", "hank williams iii", "hanoi rocks", "hanson", "hanzel und gretyl", "hardcore superstar", "harem scarem", "harry belafonte", "harry chapin", "harry james", "harry nilsson", "haste the day", "hate", "hatebreed", "haujobb", "hawk nelson", "hawksley workman", "hawkwind", "hawthorne heights", "hayden", "hayley westenra", "heart", "heather alexander", "heather dale", "heather nova", "heaven shall burn", "heideroosjes", "heinrich schiff", "helloween", "helmet", "helstar", "henry mancini", "henryk szeryng", "herbie hancock", "herman brood", "hey", "hilary duff", "hillsong", "hilltop hoods", "hocico", "hoobastank", "hoodoo gurus", "hot chocolate", "hot dad", "hot water music", "house of lords", "howard jones", "howard shore", "huntingtons", "hypocrisy", "ian anderson", "ian hunter", "ian thomas", "ice cube", "iced earth", "icehouse", "iggy azalea", "iggy pop", "ignacy jan paderewski", "igor stravinsky", "ilse delange", "immortal technique", "impaled nazarene", "in flames", "in strict confidence", "in this moment", "incantation", "incubus", "indica", "indigo girls", "infected mushroom", "information society", "inna", "insane clown posse", "instalok", "iona brown", "iq", "iration", "irene cara", "iris dement", "iron fire", "iron maiden", "isaac hayes", "israel houghton", "israel vibration", "itzhak perlman", "ivan moravec", "izzy stradlin", "ja rule", "jack bruce", "jack greene", "jack johnson", "jack savoretti", "jack white", "jackie evancho", "jackie wilson", "jackson browne", "jag panzer", "jagged edge", "jake owen", "james blunt", "james brown", "james cotton", "james mcmurtry", "james taylor", "jamey johnson", "jamie cullum", "jandek", "jane monheit", "janet jackson", "janis ian", "janis joplin", "jann arden", "jars of clay", "jascha heifetz", "jason aldean", "jason derulo", "jason mraz", "jay sean", "jean michel jarre", "jean shepard", "jedi mind tricks", "jeff beck", "jeff healey", "jefferson airplane", "jefferson starship", "jennifer lopez", "jennifer rush", "jens lekman", "jeremy camp", "jermaine jackson", "jerry butler", "jerry jeff walker", "jerry lee lewis", "jerusalem", "jesse mccartney", "jessica simpson", "jessie j", "jesus culture", "jethro tull", "jewel", "jill scott", "jill sobule", "jim brickman", "jim croce", "jim ed brown", "jim jones", "jim reeves", "jimi hendrix", "jimmie rodgers", "jimmy buffett", "jimmy cliff", "jimmy dean", "jimmy dorsey", "jimmy reed", "jimmy rushing", "jls", "jo stafford", "joan armatrading", "joan baez", "joan jett and the blackhearts", "joan of arc", "joan osborne", "jodeci", "jody watley", "joe", "joe bonamassa", "joe budden", "joe cocker", "joe diffie", "joe ely", "joe henry", "joe jackson", "joe nichols", "joe purdy", "joe walsh", "johann sebastian bach", "johannes brahms", "john anderson", "john browning", "john denver", "john entwistle", "john farnham", "john fogerty", "john hammond", "john hiatt", "john lee hooker", "john legend", "john lennon", "john martyn", "john mayall", "john mayer", "john mccormack", "john mellencamp", "john michael montgomery", "john miles", "john ogdon", "john prine", "john stewart", "john waite", "john wesley harding", "john wetton", "johnny cash", "johnny hallyday", "johnny horton", "johnny mathis", "johnny mercer", "johnny paycheck", "johnny rivers", "johnny winter", "jojo", "jon anderson", "jon secada", "jonas brothers", "jonatha brooke", "joni mitchell", "jonny lang", "jordin sparks", "jorge bolet", "jorn", "josef suk", "joseph haydn", "joseph szigeti", "josh garrels", "josh groban", "josh ritter", "josh rouse", "josh turner", "josh woodward", "joshua bell", "joss stone", "journey", "joy division", "joy electric", "judas priest", "judie tzuke", "judy collins", "judy garland", "juelz santana", "juice newton", "julian lennon", "julian lloyd webber", "julie doiron", "julio iglesias", "jump5", "june carter cash", "jungle rot", "junior wells", "justin bieber", "justin timberlake", "justin townes earle", "kaiser chiefs", "kalmah", "kamelot", "kansas", "kanye west", "kari peitsamo", "karyn white", "kasey chambers", "kaskade", "kataklysm", "katatonia", "kate nash", "kate rusby", "katharine mcphee", "katherine jenkins", "katie melua", "katy perry", "kay starr", "keith sweat", "keith urban", "keith whitley", "kelis", "kelly clarkson", "kelly price", "ken hensley", "kendrick lamar", "kenny chesney", "kenny g", "kenny loggins", "kenny rogers", "kenny wayne shepherd", "kent", "keren ann", "keri hilson", "kid cudi", "kid rock", "kidz bop", "killing joke", "killswitch engage", "kim carnes", "kim mitchell", "kim wilde", "king crimson", "king diamond", "king missile", "kingdom come", "kirk franklin", "kiss", "kitty wells", "kmfdm", "koda kumi", "kodak black", "koko taylor", "kool keith", "korn", "korpiklaani", "kotoko", "kraan", "kraftwerk", "krayzie bone", "kreator", "kris kristofferson", "krisiun", "kristin hersh", "kt tunstall", "kula shaker", "kurupt", "kylie minogue", "la toya jackson", "labyrinth", "lacrimas profundere", "lacrimosa", "lacuna coil", "lady antebellum", "lady gaga", "ladytron", "lagwagon", "laibach", "lamb", "lamb of god", "lambchop", "lana del rey", "lana lane", "lang lang", "lara fabian", "larry norman", "laura branigan", "laura fygi", "laura marling", "laura pausini", "laurie anderson", "lazar berman", "lea salonga", "lead belly", "leatherface", "lecrae", "led zeppelin", "lee aaron", "lee greenwood", "lee hazlewood", "lefty frizzell", "leif ove andsnes", "lena horne", "lenny kravitz", "leo jan ek", "leo sayer", "leon fleisher", "leon russell", "leona lewis", "leonard bernstein", "leonard cohen", "leonard pennario", "leonid kogan", "lesley gore", "less than jake", "level 42", "levon helm", "lightning seeds", "lil boosie", "lil wayne", "lil yachty", "lily allen", "limp bizkit", "linda ronstadt", "lindisfarne", "linkin park", "linton kwesi johnson", "lionel hampton", "lionel richie", "lisa ekdahl", "lit", "lita ford", "little big town", "little mix", "little richard", "little river band", "little walter", "living colour", "living sacrifice", "ll cool j", "lmfao", "lobo", "logic", "lonestar", "lonnie donegan", "lonnie johnson", "lord belial", "lord of the lost", "lordi", "lords of acid", "loreena mckennitt", "loretta lynn", "lorrie morgan", "los lobos", "los pericos", "lou rawls", "lou reed", "loudon wainwright iii", "louis armstrong", "louis jordan", "louis lortie", "lucinda williams", "lucky dube", "ludacris", "ludwig van beethoven", "luka bloom", "luke bryan", "luna", "lupe fiasco", "luther allison", "luther vandross", "lyfe jennings", "lyle lovett", "lynn anderson", "lynyrd skynyrd", "m83", "ma rainey", "maanam", "maaya sakamoto", "mac davis", "mac miller", "macabre", "machinae supremacy", "machine gun kelly", "machine head", "macy gray", "mad caddies", "madeleine peyroux", "madness", "madonna", "maggie reilly", "magnum", "mai kuraki", "majesty", "malcolm holcombe", "malevolent creation", "malvina reynolds", "man", "mandy moore", "manfred mann", "manowar", "marc almond", "marc anthony", "marc cohn", "marcelle meyer", "marcia griffiths", "marduk", "maria jo o pires", "maria mena", "maria muldaur", "mariah carey", "marillion", "marilyn manson", "marina and the diamonds", "mario", "marissa nadler", "mark chesnutt", "mark erelli", "mark heard", "mark knopfler", "mark kozelek", "mark ronson", "marlene dietrich", "martha and the vandellas", "martha argerich", "martha wainwright", "martin carthy", "martina mcbride", "marty robbins", "marvin gaye", "mary mary", "mason jennings", "massive attack", "masta ace", "master p", "mastodon", "mat kearney", "matisyahu", "matt haimovitz", "matt maher", "matt nathanson", "matt pond pa", "matt redman", "matthew good", "matthew ryan", "maureen mcgovern", "maurice ravel", "maurizio pollini", "mavis staples", "max romeo", "mc chris", "mc eiht", "mc frontalot", "mc hammer", "mcfly", "me first and the gimme gimmes", "meat loaf", "meek mill", "megadeth", "mel tillis", "melanie", "melanie c", "melissa etheridge", "melissa manchester", "melvins", "memphis may fire", "memphis minnie", "memphis slim", "menudo", "mercyful fate", "mercyme", "merle haggard", "meshuggah", "metal church", "metallica", "method man", "metric", "mew", "mewithoutyou", "mgmt", "michael bolton", "michael card", "michael franks", "michael jackson", "michael johnson", "michael learns to rock", "michael martin murphey", "michael mcdonald", "michael nesmith", "michael rabin", "michael schenker group", "mickey gilley", "mickey newbury", "migos", "mika", "mike doughty", "mike jones", "mike oldfield", "mikhail pletnev", "mildred bailey", "miley cyrus", "millencolin", "mina", "mindless self indulgence", "ministry", "mint condition", "minus the bear", "mirah", "miranda lambert", "misanthrope", "misfits", "miss may i", "mississippi fred mcdowell", "mississippi john hurt", "missy elliott", "mobb deep", "moby", "modern talking", "modest petrovich mussorgsky", "molly hatchet", "monster magnet", "montgomery gentry", "monty python", "moonspell", "morbid angel", "morcheeba", "morgana lefay", "mormon tabernacle choir", "mortification", "mos def", "mose allison", "mostly autumn", "mother mother", "motorpsycho", "mott the hoople", "moya brennan", "mud", "muddy waters", "mudvayne", "municipal waste", "murray perahia", "murs", "mushroomhead", "musiq soulchild", "mustasch", "mxpx", "my life with the thrill kill kult", "my morning jacket", "myra hess", "mystic prophecy", "mystikal", "namie amuro", "nana mouskouri", "nanci griffith", "nancy sinatra", "nancy wilson", "napalm death", "nas", "nat king cole", "natalie cole", "natalie grant", "natalie imbruglia", "natasha bedingfield", "nathan milstein", "nazareth", "neal morse", "necro", "neil diamond", "neil sedaka", "neil young", "neko case", "nekromantix", "nektar", "nellie mckay", "nelly", "nelly furtado", "nelson", "nelson freire", "nena", "nerina pallot", "neurosis", "never shout never", "nevermore", "new edition", "new found glory", "new model army", "new order", "new riders of the purple sage", "new trolls", "new york philharmonic", "newsboys", "nic jones", "nick drake", "nick lowe", "nickelback", "nicki minaj", "nicole scherzinger", "night ranger", "nightrage", "nightwish", "nik kershaw", "nikka costa", "nile", "nina hagen", "nina simone", "nine inch nails", "ninja sex party", "nirvana", "nitty gritty dirt band", "no fun at all", "no use for a name", "noa", "nocturnal rites", "nofx", "nomeansno", "nonpoint", "norah jones", "norma jean", "nouvelle vague", "nox arcana", "nuclear assault", "nunslaughter", "obie trice", "odetta", "of montreal", "ohio players", "okkervil river", "olly murs", "omnia", "one direction", "opeth", "opus", "orchestral manoeuvres in the dark", "otep", "otis redding", "otis rush", "otis spann", "our last night", "outkast", "over the rhine", "overkill", "owl city", "ozzy osbourne", "pablo casals", "pain of salvation", "pam tillis", "pansy division", "pantera", "papa roach", "paradise lost", "paragon", "parkway drive", "parliament", "parov stelar", "parry gripp", "pascal rog ", "passenger", "pat benatar", "pathology", "patsy cline", "patti labelle", "patti page", "patty griffin", "patty loveless", "paul and storm", "paul anka", "paul carrack", "paul gilbert", "paul kelly", "paul mccartney", "paul oakenfold", "paul simon", "paul tortelier", "paul van dyk", "paul wall", "paul weller", "paul whiteman", "paul wilbur", "paul williams", "paul young", "peabo bryson", "peggy lee", "pendragon", "pennywise", "pentatonix", "pepper", "per gessle", "perry como", "pet shop boys", "pete seeger", "pete townshend", "peter and the test tube babies", "peter andre", "peter frampton", "peter gabriel", "peter green", "peter green splinter group", "peter hammill", "peter serkin", "peter tosh", "peter wolf", "petra", "petula clark", "phantom planet", "pharrell williams", "phil collins", "phil harris", "phil keaggy", "phil ochs", "phil wickham", "philadelphia orchestra", "philip glass", "philippe entremont", "piebald", "pierre fournier", "pierre laurent aimard", "pieter wispelwey", "pig", "pig destroyer", "pinback", "pink cream 69", "pink floyd", "pitbull", "pitchshifter", "planetshakers", "plumb", "poco", "poison", "polysics", "porter wagoner", "powerman 5000", "powerwolf", "prefab sprout", "pretenders", "primal fear", "primus", "prince", "procol harum", "project 86", "project pat", "project pitchfork", "prong", "propagandhi", "psyche", "psychostick", "public enemy", "puffy amiyumi", "puhdys", "pulp", "punchline", "pyotr ilyich tchaikovsky", "qntal", "quasi", "queen", "quiet riot", "racoon", "radu lupu", "raekwon", "raffi", "rage", "raimon", "ralph mctell", "ramones", "rancid", "randy newman", "randy rogers band", "randy stonehill", "randy travis", "rascal flatts", "rasputina", "ratt", "raul seixas", "ray charles", "ray davies", "ray lamontagne", "ray wylie hubbard", "rbd", "rebellion", "rebelution", "red flag", "reel big fish", "regina spektor", "rehab", "renaissance", "renard", "resurrection band", "reverend gary davis", "reverend horton heat", "revocation", "rhapsody of fire", "rheostatics", "rhonda vincent", "rich mullins", "richard goode", "richard strauss", "richard thompson", "richard wagner", "richie kotzen", "richie spice", "richmond fontaine", "rick astley", "rick ross", "rick springfield", "rick wakeman", "ricky martin", "ricky nelson", "ricky skaggs", "rihanna", "ringo starr", "riot", "rise against", "riverside", "robben ford", "robbie williams", "robert casadesus", "robert cray", "robert earl keen", "robert johnson", "robert palmer", "robert plant", "robert pollard", "robert schuman", "robert wyatt", "roberta flack", "robin trower", "robyn", "roch voisine", "rocky votolato", "rod stewart", "rodney crowell", "roger daltrey", "roger mcguinn", "roger miller", "roger waters", "roger whittaker", "rome", "ron sexsmith", "ronan keating", "ronnie milsap", "rory block", "rory gallagher", "rosalyn tureck", "rosanne cash", "rose royce", "rosemary clooney", "rotten sound", "rotting christ", "roxette", "roxy music", "roy acuff", "roy buchanan", "roy clark", "roy drusky", "roy orbison", "royal hunt", "rudimentary peni", "rudolf firku n ", "rudolf serkin", "rufus", "rufus wainwright", "running wild", "runrig", "rupaul", "rush", "ry cooder", "ryan adams", "rza", "s club 7", "sabaton", "sade", "saga", "sage francis", "said the whale", "saint etienne", "sam cooke", "sam phillips", "sam smith", "samael", "samantha fox", "sammie", "sammy hagar", "samson fran ois", "san francisco symphony", "sanctus real", "sandie shaw", "sandra", "sandy denny", "santana", "sara bareilles", "sara evans", "sarah brightman", "sarah connor", "sarah mclachlan", "sarah vaughan", "sasha", "savatage", "saves the day", "savoy brown", "sawyer brown", "saxon", "scarface", "schiller", "scooter", "scorpions", "scott joplin", "screaming trees", "screeching weasel", "seal", "sean paul", "sebadoh", "secret garden", "selena", "selena gomez", "senses fail", "sentenced", "sergei prokofiev", "sergei rachmaninoff", "sesame street", "sevendust", "seventh day slumber", "sex pistols", "shadows fall", "shaggy", "shakira", "shalamar", "sham 69", "shania twain", "shawn colvin", "shawn mendes", "shearwater", "sheena easton", "shel silverstein", "shelby lynne", "sheryl crow", "shinee", "shirley bassey", "shirley horn", "shlomo mintz", "shocking blue", "shooter jennings", "showaddywaddy", "showbread", "shura cherkassky", "sick of it all", "sigh", "silkk the shocker", "silverstein", "simon and garfunkel", "simple minds", "simply red", "sinister", "sinner", "sirenia", "sister hazel", "sister sledge", "six feet under", "skeeter davis", "skid row", "skinny puppy", "skip james", "skrillex", "skyclad", "skylark", "slade", "slayer", "slightly stoopid", "slim thug", "smokey robinson", "smokie", "smoking popes", "snfu", "snog", "snoop dogg", "sodom", "soft machine", "sol invictus", "solomon cutner", "son house", "son volt", "sonata arctica", "sondre lerche", "sonny boy williamson i", "sonny boy williamson ii", "sonny james", "sonny terry", "sophie zelmani", "sopor aeternus", "soulfly", "soulja boy", "spandau ballet", "sparks", "spice 1", "split enz", "spoon", "squeeze", "stacey kent", "stan ridgway", "stan rogers", "starflyer 59", "stars", "status quo", "steam powered giraffe", "steeleye span", "steely dan", "stephan eicher", "stephen malkmus", "stephen sondheim", "stephen stills", "steppenwolf", "steps", "stereolab", "steve earle", "steve forbert", "steve goodman", "steve green", "steve hackett", "steve miller band", "steve vai", "steve wariner", "steve winwood", "steven curtis chapman", "steven isserlis", "steven wilson", "stevie nicks", "stevie ray vaughan", "stevie wonder", "stick to your guns", "stiff little fingers", "sting", "stonewall jackson", "stratovarius", "strawbs", "stray cats", "stryper", "styles p", "styx", "sublime", "suburban legends", "sufjan stevens", "sugababes", "sugarland", "suicidal tendencies", "suicide commando", "suidakra", "sun kil moon", "super junior", "superbus", "supertramp", "survivor", "susan boyle", "suzanne vega", "suzi quatro", "suzy bogguss", "sviatoslav richter", "swans", "sweet", "sweetbox", "swollen members", "swv", "symphony x", "tad morose", "taj mahal", "take 6", "take that", "talib kweli", "talking heads", "tall dwarfs", "tammy wynette", "tampa red", "tanita tikaram", "tank", "tankard", "tanya tucker", "tarja", "tarot", "taylor dayne", "taylor swift", "team starkid", "tears for fears", "tech n9ne", "ted leo and the pharmacists", "ted nugent", "teena marie", "ten years after", "tennessee ernie ford", "tenth avenue north", "teresa brewer", "terminal choice", "terri clark", "terrorgruppe", "testament", "tex ritter", "tha dogg pound", "the 5th dimension", "the 69 eyes", "the acacia strain", "the adicts", "the alan parsons project", "the allman brothers band", "the ames brothers", "the amity affliction", "the andrews sisters", "the angels", "the animals", "the appleseed cast", "the arrogant worms", "the association", "the avett brothers", "the band", "the bangles", "the baseballs", "the beach boys", "the beatles", "the beau brummels", "the beautiful south", "the bellamy brothers", "the birthday massacre", "the black crowes", "the black dahlia murder", "the black eyed peas", "the black keys", "the bottle rockets", "the bouncing souls", "the boys", "the brian setzer orchestra", "the bronx", "the brothers four", "the browns", "the busters", "the byrds", "the cars", "the carter family", "the casualties", "the charlie daniels band", "the chemical brothers", "the chieftains", "the choir", "the chordettes", "the church", "the clash", "the clientele", "the corrs", "the coup", "the cramps", "the cross movement", "the crown", "the damned", "the darkness", "the derek trucks band", "the devil wears prada", "the dickies", "the dillinger escape plan", "the diplomats", "the divine comedy", "the donnas", "the doobie brothers", "the doors", "the drifters", "the dubliners", "the echoing green", "the everly brothers", "the fabulous thunderbirds", "the faint", "the felice brothers", "the fiery furnaces", "the fixx", "the flower kings", "the four aces", "the four lads", "the four seasons", "the frames", "the front bottoms", "the game", "the gathering", "the grass roots", "the guess who", "the handsome family", "the haunted", "the hellacopters", "the hold steady", "the hollies", "the human league", "the impressions", "the incredible string band", "the ink spots", "the irish rovers", "the isley brothers", "the jackson 5", "the jam", "the jayhawks", "the jimi hendrix experience", "the judds", "the kelly family", "the kills", "the kingston trio", "the kinks", "the kooks", "the lads", "the lawrence arms", "the locust", "the lucksmiths", "the magnetic fields", "the maine", "the manhattan transfer", "the manhattans", "the marvelettes", "the mavericks", "the mccalmans", "the meteors", "the mighty mighty bosstones", "the mission", "the moffatts", "the monkees", "the monochrome set", "the moody blues", "the muppets", "the national", "the new pornographers", "the nits", "the nylons", "the oak ridge boys", "the offspring", "the osmonds", "the outfield", "the partridge family", "the pillows", "the pineapple thief", "the platters", "the pogues", "the pointer sisters", "the police", "the poodles", "the posies", "the presidents of the united states of america", "the pretty things", "the proclaimers", "the prodigy", "the psychedelic furs", "the pussycat dolls", "the rascals", "the rasmus", "the raveonettes", "the real mckenzies", "the replacements", "the residents", "the righteous brothers", "the roches", "the rolling stones", "the roots", "the saints", "the samples", "the saturdays", "the saw doctors", "the script", "the searchers", "the seatbelts", "the seekers", "the shadows", "the shins", "the shirelles", "the smashing pumpkins", "the smithereens", "the smiths", "the soundtrack of our lives", "the spinners", "the stanley brothers", "the stone roses", "the stranglers", "the streets", "the string cheese incident", "the strokes", "the style council", "the stylistics", "the supremes", "the tea party", "the temptations", "the the", "the tragically hip", "the tremeloes", "the triffids", "the troggs", "the tubes", "the turtles", "the undertones", "the unseen", "the used", "the vandals", "the velvet underground", "the ventures", "the veronicas", "the verve", "the verve pipe", "the vines", "the violet burning", "the walkabouts", "the walker brothers", "the walkmen", "the wallflowers", "the wanted", "the waterboys", "the weeknd", "the whispers", "the white buffalo", "the white stripes", "the who", "the wiggles", "the wildhearts", "the wonder years", "the yardbirds", "the zombies", "thea gilmore", "theatre of tragedy", "theatres des vampires", "thee oh sees", "therion", "they might be giants", "thievery corporation", "thin lizzy", "third day", "third eye blind", "third world", "thompson twins", "thousand foot krutch", "three 6 mafia", "three dog night", "threshold", "thrice", "throwdown", "thunder", "thursday", "tiamat", "tim hughes", "tim maia", "tim mcgraw", "timbaland", "tina arena", "tina dico", "tina turner", "tinashe", "tindersticks", "tinie tempah", "tism", "tlc", "tnt", "toad the wet sprocket", "toby keith", "tobymac", "today is the day", "todd rundgren", "todd snider", "tokio hotel", "tom jones", "tom mcrae", "tom paxton", "tom petty and the heartbreakers", "tom rosenthal", "tom russell", "tom waits", "tommy dorsey", "tommy roe", "toni braxton", "tony banks", "tony bennett", "tony joe white", "tori amos", "toro y moi", "toto", "tourniquet", "townes van zandt", "toy dolls", "toyah", "tracy byrd", "tracy chapman", "tracy lawrence", "trae", "traffic", "trail of tears", "train", "transit", "transmetal", "trashcan sinatras", "travis", "travis scott", "trey songz", "trick daddy", "tricky", "trip lee", "tripod", "trisha yearwood", "tristania", "triumph", "trivium", "trooper", "trophy scars", "trouble", "tru", "truls m rk", "trust", "tsol", "tub ring", "twenty one pilots", "twila paris", "twista", "twisted sister", "twiztid", "ty segall", "type o negative", "tyrese", "tyrone wells", "u2", "ub40", "ufo", "ugk", "ultravox", "ulver", "uncle dave macon", "underoath", "underworld", "unkle", "unwritten law", "uriah heep", "useless id", "usher", "utada hikaru", "vader", "vampire rodents", "van cliburn", "van der graaf generator", "van halen", "van morrison", "vanden plas", "vanessa williams", "vangelis", "vanilla ice", "vast", "veggietales", "velvet acid christ", "venom", "vera lynn", "vern gosdin", "vertical horizon", "veruca salt", "vic chesnutt", "vic damone", "vicious rumors", "vienna philharmonic", "vigilantes of love", "viktoria mullova", "vince gill", "vinnie paz", "violent femmes", "virgin steele", "vision divine", "vladimir ashkenazy", "vladimir horowitz", "vladimir sofronitsky", "vnv nation", "voivod", "voltaire", "vonda shepard", "wale", "waltari", "walter gieseking", "walter trout", "wanda jackson", "war", "warrant", "warren zevon", "watch tower bible and tract society", "waylon jennings", "we the kings", "webb pierce", "webb wilder", "ween", "weezer", "weird al yankovic", "westlife", "whiskeytown", "white denim", "white heart", "whitechapel", "whitecross", "whitesnake", "whitney houston", "widespread panic", "wilco", "wilhelm backhaus", "wilhelm kempff", "will smith", "will young", "william fitzsimmons", "willie dixon", "willie nelson", "willy deville", "wilson phillips", "wilson pickett", "wings", "wire", "wise guys", "wishbone ash", "within temptation", "wizard", "wolfe tones", "wolfgang amadeus mozart", "wolfgang schneiderhan", "wolfgun", "woody guthrie", "wovenhand", "wumpscut", "wyclef jean", "wynn stewart", "wynonna judd", "x", "xandria", "xiu xiu", "xtc", "xv", "xxxtentacion", "xzibit", "yann tiersen", "yefim bronfman", "yehudi menuhin", "yello", "yellowcard", "yellowman", "yendri", "yes", "yngwie malmsteen", "yo la tengo", "yo yo ma", "yoko ono", "young jeezy", "young thug", "youri egorov", "zac brown band", "zachary richard"]
select_genre = "No Preference"#@param  ["No Preference", "acoustic", "ambient", "americana", "bluegrass", "blues", "britpop", "celtic", "chanson", "christian", "christmas", "classical", "comedy", "country", "crossover", "dance", "deathcore", "deutschrock", "disco", "ebm", "electro", "electronic", "electronica", "electropop", "emo", "eurodance", "europop", "experimental", "fado", "folk", "funk", "gospel", "gothic", "grindcore", "grunge", "horrorcore", "house", "industrial", "jazz", "latin", "medieval", "metalcore", "movie", "mpb", "musical", "n", "neofolk", "pop", "psychedelic", "reggae", "reggaeton", "rock", "rockabilly", "salsa", "samba", "schlager", "sertanejo", "ska", "soul", "soundtrack", "swing", "synthpop", "tejano", "television", "trance", "vocal", "worship"]

if select_artist == "No Preference":
  select_artist = "unknown"
if select_genre == "No Preference":
  select_genre = "unknown"

#@markdown ---

#START GDRIVE MODEL LOADER
import os.path
!apt install pv
!mkdir /root/.cache ; mkdir /root/.cache/jukebox ; mkdir /root/.cache/jukebox/models ; mkdir /root/.cache/jukebox/models/1b_lyrics ; mkdir /root/.cache/jukebox/models/5b_lyrics ; mkdir /root/.cache/jukebox/models/5b
!mkdir /content/gdrive/MyDrive/jukebox
!mkdir /content/gdrive/MyDrive/jukebox/models
!mkdir /content/gdrive/MyDrive/jukebox/models/5b
!mkdir /content/gdrive/MyDrive/jukebox/models/5b_lyrics
!mkdir /content/gdrive/MyDrive/jukebox/models/1b_lyrics


def load_5b_vqvae():
    if os.path.exists("/root/.cache/jukebox/models/5b/vqvae.pth.tar") == False:
      if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar") == False:
        print("5b_vqvae not stored in Google Drive. Downloading for the first time.")
        !wget https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar
      else:
        print("5b_vqvae stored in Google Drive.")
      print('Copying 5b VQVAE')
      !pv /content/gdrive/MyDrive/jukebox/models/5b/vqvae.pth.tar > /root/.cache/jukebox/models/5b/vqvae.pth.tar

def load_1b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar") == False:
      print("1b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar
    else:
      print("1b_lyrics_level_2 stored in Google Drive.")
    print("Copying 1B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/1b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar

def load_5b_lyrics_level2():
  if os.path.exists("/root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar") == False:
      print("5b_lyrics_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar
    else:
      print("5b_lyrics_level_2 stored in Google Drive.")
    print("Copying 5B_Lyrics Level 2")
    !pv /content/gdrive/MyDrive/jukebox/models/5b_lyrics/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar

def load_5b_level1():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_1.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar") == False:
      print("5b_level_1 not stored in Google Drive. Downloading for the first time. This may take a few more minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar
    else:
      print("5b_level_1 stored in Google Drive.")
    print("Copying 5B Level 1")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_1.pth.tar > /root/.cache/jukebox/models/5b/prior_level_1.pth.tar

def load_5b_level0():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_0.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar") == False:
      print("5b_level_0 not stored in Google Drive. Downloading for the first time. This may take a few minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar
    else:
      print("5b_level_0 stored in Google Drive.")
    print("Copying 5B Level 0")
    !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_0.pth.tar > /root/.cache/jukebox/models/5b/prior_level_0.pth.tar

def load_5b_level2():
  if os.path.exists('/root/.cache/jukebox/models/5b/prior_level_2.pth.tar') == False:
    if os.path.exists("/content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar") == False:
      print("5b_level_2 not stored in Google Drive. Downloading for the first time. This will take up to 10-15 minutes.")
      !wget https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_2.pth.tar -O /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar
    else:
      print("5b_level_2 stored in Google Drive.")
  print("Copying 5B Level 2")
  !pv /content/gdrive/MyDrive/jukebox/models/5b/prior_level_2.pth.tar > /root/.cache/jukebox/models/5b/prior_level_2.pth.tar

if save_and_load_models_from_drive == True:
  if model == '5b_lyrics':
    load_5b_vqvae()
    load_5b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
  if model == '5b':
    load_5b_vqvae()
    load_5b_level2()
    load_5b_level1()
    load_5b_level0()
  elif model == '1b_lyrics':
    load_5b_vqvae()
    load_1b_lyrics_level2()
    load_5b_level1()
    load_5b_level0()
#END GDRIVE MODEL LOADER

#@markdown The location for the save directory and audio file start at the root directory of your Google Drive. While you may recurse into subdirectories, do not add a "/" at the beginning or end of either field.

hps = Hyperparams()
hps.sr = 44100

#START OPTIONAL SAMPLE DOUBLER
##if gpu_info[0].find('A100') >= 0:
##  hps.n_samples =  2
##  print('A100 detected! You will get two audio samples automatically.')
##else:
##  hps.n_samples =  1
#END OPTIONAL SAMPLE DOUBLER
hps.n_samples =  1


# Specifies the directory to save the sample in.
# We set this to the Google Drive mount point.
save_directory = 'project_name' #@param {type:"string"}
full_save_directory = '/content/gdrive/MyDrive/'+save_directory
save_directory = full_save_directory
hps.name = save_directory
chunk_size = 1024 if model in ('5b', '5b_lyrics') else 32
hps.levels = 3


vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 1048576)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# The default mode of operation.
# Creates songs based on artist and genre conditioning.
mode = 'primed'
if mode == 'ancestral':
  codes_file=None
  audio_file=None
  prompt_length_in_seconds=None
if mode == 'primed':
  codes_file=None
  # Specify an audio file here.
  audio_file = 'audio_file.wav' #@param {type:"string"}
  full_audio_file = '/content/gdrive/MyDrive/'+audio_file
  audio_file = full_audio_file
  # Specify how many seconds of audio to prime on.
  #prompt_length_in_seconds = 23 (THIS IS OLD AND REPLACED BELOW)

round_primer_duration_down_to_nearest_integer = True #param {type:"boolean"}

remove_vocals_from_input = False #@param {type:"boolean"}

generation_length_in_seconds = "29"#@param[29,46,59,89,118]

#@markdown ---

rendering_quality = "Normal" #@param ["Fast", "Normal", "High-Quality", "BPM", "BPMx2"]
upsampling_quality = "Fast" #@param ["Super-Fast (Low-Quality)", "Fast", "High-Quality", "BPM"]

if upsampling_quality == "Super-Fast (Low-Quality)":
  if generation_length_in_seconds == "59":
    print("SUPER FAST UPSAMPLING SELECTED. ROUNDING GENERATION TIME DOWN TO 46.")
    generation_length_in_seconds = "46"
  elif generation_length_in_seconds == "118":
    print("SUPER FAST UPSAMPLING SELECTED. ROUNDING GENERATION TIME DOWN TO 94.")
    generation_length_in_seconds = "94"
else:
  if generation_length_in_seconds == "46":
    print("SUPER FAST UPSAMPLING NOT SELECTED. ROUNDING GENERATION TIME DOWN TO 29.")
    generation_length_in_seconds = "29"
  elif generation_length_in_seconds == 94:
    print("SUPER FAST UPSAMPLING NOT SELECTED. ROUNDING GENERATION TIME DOWN TO 89.")
    generation_length_in_seconds = "89"


#PROCESS AUDIO TO WAV
import os
import sys
!apt install sox ffmpeg
os.system("ffmpeg -y -i "+audio_file+" -vn -sn -t 00:00:23.001 -c:a pcm_s24le -ac 2 -ar 44100 /content/pre-primer.wav")

if remove_vocals_from_input == True:
  !pip install demucs
  !mkdir /content/removevox/
  !mkdir /content/removevox_result/
  !cp /content/pre-primer.wav /content/removevox/pre-primer.wav
  # Customize the following options!
  model = "mdx_extra"
  extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
  #two_stems = None   # only separate one stems from the rest, for instance
  two_stems = "vocals"

  # Options for the output audio.
  mp3 = False
  mp3_rate = 320
  float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
  int24 = True    # output as int24 wavs, unused if 'mp3' is True.
  # You cannot set both `float32 = True` and `int24 = True` !!
  #@title Useful functions, don't forget to execute
  in_path = '/content/removevox/'
  out_path = '/content/removevox_result/'
  import io
  from pathlib import Path
  import select
  from shutil import rmtree
  import subprocess as sp
  import sys
  from typing import Dict, Tuple, Optional, IO

  from google.colab import files

  def find_files(in_path):
      out = []
      for file in Path(in_path).iterdir():
          if file.suffix.lower().lstrip(".") in extensions:
              out.append(file)
      return out

  def copy_process_streams(process: sp.Popen):
      def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
          assert stream is not None
          if isinstance(stream, io.BufferedIOBase):
              stream = stream.raw
          return stream

      p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
      stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
          p_stdout.fileno(): (p_stdout, sys.stdout),
          p_stderr.fileno(): (p_stderr, sys.stderr),
      }
      fds = list(stream_by_fd.keys())

      while fds:
          # `select` syscall will wait until one of the file descriptors has content.
          ready, _, _ = select.select(fds, [], [])
          for fd in ready:
              p_stream, std = stream_by_fd[fd]
              raw_buf = p_stream.read(2 ** 16)
              if not raw_buf:
                  fds.remove(fd)
                  continue
              buf = raw_buf.decode()
              std.write(buf)
              std.flush()

  def separate(inp=None, outp=None):
      inp = inp or in_path
      outp = outp or out_path
      cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
      if mp3:
          cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
      if float32:
          cmd += ["--float32"]
      if int24:
          cmd += ["--int24"]
      if two_stems is not None:
          cmd += [f"--two-stems={two_stems}"]
      files = [str(f) for f in find_files(inp)]
      if not files:
          print(f"No valid audio files in {in_path}")
          return
      print("Going to separate the files:")
      print('\n'.join(files))
      print("With command: ", " ".join(cmd))
      p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
      copy_process_streams(p)
      p.wait()
      if p.returncode != 0:
          print("Command failed, something went wrong.")


  def from_upload():
      out_path = Path('separated')
      in_path = Path('tmp_in')
      
      if in_path.exists():
          rmtree(in_path)
      in_path.mkdir()
      
      if out_path.exists():
          rmtree(out_path)
      out_path.mkdir()
      
      uploaded = files.upload()
      for name, content in uploaded.items():
          (in_path / name).write_bytes(content)
      separate(in_path, out_path)

  separate()

  !mv /content/removevox_result/mdx_extra/pre-primer/no_vocals.wav /content/pre-primer.wav
  !rm -r /content/removevox/ /content/removevox_result/

os.system("sox /content/pre-primer.wav -c 1 -b 16 -e signed /content/primer.wav trim 0 23 norm -1.2")
audio_file = "/content/primer.wav"

if mode == 'primed':
  if round_primer_duration_down_to_nearest_integer == False:
    prompt_length_in_seconds = int(librosa.get_duration(filename=f'/content/primer.wav'))
  else:
    prompt_length_in_seconds = math.floor(int(librosa.get_duration(filename=f'/content/primer.wav')))

#START BPM DETECT

if rendering_quality == "BPM" or rendering_quality == "BPMx2" or upsampling_quality == "BPM":
  !pip3 install pywavelets matplotlib
    # Copyright 2012 Free Software Foundation, Inc.
  #
  # This file is part of The BPM Detector Python
  #
  # The BPM Detector Python is free software; you can redistribute it and/or modify
  # it under the terms of the GNU General Public License as published by
  # the Free Software Foundation; either version 3, or (at your option)
  # any later version.
  #
  # The BPM Detector Python is distributed in the hope that it will be useful,
  # but WITHOUT ANY WARRANTY; without even the implied warranty of
  # MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
  # GNU General Public License for more details.
  #
  # You should have received a copy of the GNU General Public License
  # along with The BPM Detector Python; see the file COPYING.  If not, write to
  # the Free Software Foundation, Inc., 51 Franklin Street,
  # Boston, MA 02110-1301, USA.

  import argparse
  import array
  import math
  import wave

  import matplotlib.pyplot as plt
  import numpy
  import pywt
  from scipy import signal

  os.system("sox "+audio_file+" -c 2 -b 16 -e signed /content/bpmdetect.wav rate -va 44100")

  def read_wav(filename):
      # open file, get metadata for audio
      try:
          wf = wave.open(filename, "rb")
      except IOError as e:
          print(e)
          return

      # typ = choose_type( wf.getsampwidth() ) # TODO: implement choose_type
      nsamps = wf.getnframes()
      assert nsamps > 0

      fs = wf.getframerate()
      assert fs > 0

      # Read entire file and make into an array
      samps = list(array.array("i", wf.readframes(nsamps)))

      try:
          assert nsamps == len(samps)
      except AssertionError:
          print(nsamps, "not equal to", len(samps))

      return samps, fs


  # print an error when no data can be found
  def no_audio_data():
      print("No audio data for sample, skipping...")
      return None, None


  # simple peak detection
  def peak_detect(data):
      max_val = numpy.amax(abs(data))
      peak_ndx = numpy.where(data == max_val)
      if len(peak_ndx[0]) == 0:  # if nothing found then the max must be negative
          peak_ndx = numpy.where(data == -max_val)
      return peak_ndx


  def bpm_detector(data, fs):
      cA = []
      cD = []
      correl = []
      cD_sum = []
      levels = 4
      max_decimation = 2 ** (levels - 1)
      min_ndx = math.floor(60.0 / 220 * (fs / max_decimation))
      max_ndx = math.floor(60.0 / 40 * (fs / max_decimation))

      for loop in range(0, levels):
          cD = []
          # 1) DWT
          if loop == 0:
              [cA, cD] = pywt.dwt(data, "db4")
              cD_minlen = len(cD) / max_decimation + 1
              cD_sum = numpy.zeros(math.floor(cD_minlen))
          else:
              [cA, cD] = pywt.dwt(cA, "db4")

          # 2) Filter
          cD = signal.lfilter([0.01], [1 - 0.99], cD)

          # 4) Subtract out the mean.

          # 5) Decimate for reconstruction later.
          cD = abs(cD[:: (2 ** (levels - loop - 1))])
          cD = cD - numpy.mean(cD)

          # 6) Recombine the signal before ACF
          #    Essentially, each level the detail coefs (i.e. the HPF values) are concatenated to the beginning of the array
          cD_sum = cD[0 : math.floor(cD_minlen)] + cD_sum

      if [b for b in cA if b != 0.0] == []:
          return no_audio_data()

      # Adding in the approximate data as well...
      cA = signal.lfilter([0.01], [1 - 0.99], cA)
      cA = abs(cA)
      cA = cA - numpy.mean(cA)
      cD_sum = cA[0 : math.floor(cD_minlen)] + cD_sum

      # ACF
      correl = numpy.correlate(cD_sum, cD_sum, "full")

      midpoint = math.floor(len(correl) / 2)
      correl_midpoint_tmp = correl[midpoint:]
      peak_ndx = peak_detect(correl_midpoint_tmp[min_ndx:max_ndx])
      if len(peak_ndx) > 1:
          return no_audio_data()

      peak_ndx_adjusted = peak_ndx[0] + min_ndx
      bpm = 60.0 / peak_ndx_adjusted * (fs / max_decimation)
      print(bpm)
      return bpm, correl

  samps, fs = read_wav("/content/bpmdetect.wav")
  data = []
  correl = []
  bpm = 0
  n = 0
  nsamps = len(samps)
  window_samps = int(3 * fs)
  samps_ndx = 0  # First sample in window_ndx
  max_window_ndx = math.floor(nsamps / window_samps)
  bpms = numpy.zeros(max_window_ndx)

  # Iterate through all windows
  for window_ndx in range(0, max_window_ndx):

      # Get a new set of samples
      # print(n,":",len(bpms),":",max_window_ndx_int,":",fs,":",nsamps,":",samps_ndx)
      data = samps[samps_ndx : samps_ndx + window_samps]
      if not ((len(data) % window_samps) == 0):
          raise AssertionError(str(len(data)))

      bpm, correl_temp = bpm_detector(data, fs)
      if bpm is None:
          continue
      bpms[window_ndx] = bpm
      correl = correl_temp

      # Iterate at the end of the loop
      samps_ndx = samps_ndx + window_samps

      # Counter for debug...
      n = n + 1

  bpm = round(numpy.median(bpms), 0)
  print("Completed!  Estimated Beats Per Minute:", bpm)
  print("If this BPM is not correct, DON'T PANIC, it's fine, it just needs to be a rough estimate or a multiple of the real BPM.")

  n = range(0, len(correl))
  plt.plot(n, abs(correl))
  plt.show(block=True)

  !rm /content/bpmdetect.wav
  bpm_fraction_l2 = calclevelrenderbpm(bpm,4.2,128)
  bpm_fraction_l2_1b = calclevelrenderbpm_1b(bpm,4.2,128)
  bpm_fraction_l1 = calclevelconditionbpm(bpm,3,32)
  bpm_fraction_l0 = calclevelconditionbpm(bpm,0.5,8)
  #END BPM DETECT

sampling_aggressiveness = "Normal" #@param ["Ultra-Low", "Very-Low", "Low", "Normal", "High", "Very-High", "Extreme"]

if sampling_aggressiveness == "Ultra-Low":
  sampling_temperature = .970
elif sampling_aggressiveness == "Very-Low":
  sampling_temperature = .972
elif sampling_aggressiveness == "Low":
  sampling_temperature = .974
elif sampling_aggressiveness == "Normal":
  sampling_temperature = .976
elif sampling_aggressiveness == "High":
  sampling_temperature = .978
elif sampling_aggressiveness == "Very-High":
  sampling_temperature = .980
elif sampling_aggressiveness == "Extreme":
  sampling_temperature = .982

if rendering_quality == "Fast":
  if model == "1b_lyrics":
    l2_selected_fraction = 0.269531
    sampling_temperature = sampling_temperature-0.002
  else:
    l2_selected_fraction = 0.359375
    sampling_temperature = sampling_temperature-0.002
elif rendering_quality == "Normal":
  l2_selected_fraction = 0.125
elif rendering_quality == "High-Quality":
  l2_selected_fraction = 0.09375
  sampling_temperature = sampling_temperature+0.002
elif rendering_quality == "BPM":
  if model == "1b_lyrics":
    l2_selected_fraction = bpm_fraction_l2_1b
    sampling_temperature = sampling_temperature+0.002
  else:
    l2_selected_fraction = bpm_fraction_l2
    sampling_temperature = sampling_temperature+0.002
elif rendering_quality == "BPMx2":
  if model == "1b_lyrics":
    l2_selected_fraction = bpm_fraction_l2_1b/2
    sampling_temperature = sampling_temperature+0.004
  else:
    l2_selected_fraction = bpm_fraction_l2/2
    sampling_temperature = sampling_temperature+0.004

if upsampling_quality == "Super-Fast (Low-Quality)":
  l1_selected_fraction = 4
  l0_selected_fraction = 2
  l1_temp = 0.97
  l0_temp = 0.97
elif upsampling_quality == "Fast":
  l1_selected_fraction = 1
  l0_selected_fraction = 1
  l1_temp = 0.99
  l0_temp = 0.99
elif upsampling_quality == "High-Quality":
  l1_selected_fraction = 0.8125
  l0_selected_fraction = 1
  l1_temp = 0.99
  l0_temp = 0.99
elif upsampling_quality == "BPM":
  l1_selected_fraction = bpm_fraction_l1
  l0_selected_fraction = 1
  l1_temp = 0.99
  l0_temp = 0.99

hps.hop_fraction = [l0_selected_fraction,l1_selected_fraction,l2_selected_fraction]

print("SET HPS.HOP_FRACTION TO: "+str(hps.hop_fraction))

sample_length_in_seconds = calcrenderduration(int(prompt_length_in_seconds),int(generation_length_in_seconds))

#@markdown ---
#@markdown JuicyJukebox Diffusion (BETA)
diffuse_audio = True #@param {type: "boolean"}
diffusion_strength = "Soften" #@param ["Soften", "Very-Mild", "Mild", "Normal", "Moderate", "Strong", "Extreme", "Insane"]
diffusion_quality = "Super-Fast (Low-Quality)" #@param ["Super-Fast (Low-Quality)", "Fast", "Normal", "High-Quality", "Extreme", "Insane"]
#@markdown Provides a more realistic sound and generates a stereo output.

if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      codes_file = data
      if int(sample_length_in_seconds) > int(librosa.get_duration(filename=f'{hps.name}/level_2/item_0.wav')):
        mode = 'continue'
      else:
        mode = 'upsample'
      break

print('mode is now '+mode)
if mode == 'continue':
  print('Continuing from level 2')
if mode == 'upsample':
  print('Upsampling from level '+str(level))

sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

if mode == 'upsample':
  sample_length_in_seconds=int(librosa.get_duration(filename=f'{hps.name}/level_{level}/item_0.wav'))
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

if mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  hps.n_samples = zs[-1].shape[0]

hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

# Note: Metas can contain different prompts per sample.
# By default, all samples use the same prompt.

metas = [dict(artist = select_artist,
            genre = select_genre,
            total_length = hps.sample_length,
            offset = 0,
            lyrics = your_lyrics, 
            ),
          ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]

if upsampling_quality == "Super-Fast (Low-Quality)":
  if gpu_info[0].find('Tesla T4') >= 0:
    lower_batch_size = 24
    print('Tesla T4 detected, lower_batch_size set to 24')
  elif gpu_info[0].find('Tesla K80') >= 0:
    lower_batch_size = 16
    print('Tesla K80 detected, lower_batch_size set to 16')
  elif gpu_info[0].find('Tesla P100') >= 0:
    lower_batch_size = 32
    print('Tesla P100 detected, lower_batch_size set to 32')
  elif gpu_info[0].find('Tesla V100') >= 0:
    lower_batch_size = 32
    print('Tesla V100 detected, lower_batch_size set to 32')
  elif gpu_info[0].find('A100') >= 0:
    lower_batch_size = 64
    print('A100 detected! Party Time!!! lower_batch_size set to 64')
  else:
    lower_batch_size = 32
    print('Different GPU detected, lower_batch_size set to 32.')
else:
  if gpu_info[0].find('Tesla T4') >= 0:
    lower_batch_size = 15
    print('Tesla T4 detected, lower_batch_size set to 15')
  elif gpu_info[0].find('Tesla K80') >= 0:
    lower_batch_size = 10
    print('Tesla K80 detected, lower_batch_size set to 10')
  elif gpu_info[0].find('Tesla P100') >= 0:
    lower_batch_size = 20
    print('Tesla P100 detected, lower_batch_size set to 20')
  elif gpu_info[0].find('Tesla V100') >= 0:
    lower_batch_size = 20
    print('Tesla V100 detected, lower_batch_size set to 20')
  elif gpu_info[0].find('A100') >= 0:
    lower_batch_size = 40
    print('A100 detected! Party Time!!! lower_batch_size set to 40')
  else:
    lower_batch_size = 20
    print('Different GPU detected, lower_batch_size set to 20.')

lower_level_chunk_size = 2048 if model in ('5b', '5b_lyrics') else 48
chunk_size = 1024 if model in ('5b', '5b_lyrics') else 32
sampling_kwargs = [dict(temp=l0_temp, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=l1_temp, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True, 
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cpu') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cpu() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('One click upsampling!')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')

# Set this False if you are on a local machine that has enough memory (this allows you to do the
# lyrics alignment visualization during the upsampling stage). For a hosted runtime, 
# we'll need to go ahead and delete the top_prior if you are using the 5b_lyrics model.
if True:
  del top_prior
  empty_cache()
  top_prior=None
upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)

if upsampling_quality == "Super-Fast (Low-Quality)":
  os.system("mv "+save_directory+"/level_0/item_0_orig.wav "+save_directory+"/level_0/item_0.wav")
  os.system("mv "+save_directory+"/level_0/item_0.wav "+save_directory+"/level_0/item_0_orig.wav")
  os.system("sox "+save_directory+"/level_0/item_0_orig.wav -c 1 -b 16 -e signed "+save_directory+"/level_0/item_0.wav sinc -n 32767 8600-8200 sinc -n 32767 13850-13750 sinc -n 32767 5550-5450")

if diffuse_audio == False:
  print("JuicyJukebox Complete! Look for your render at: "+save_directory+"/level_0/item_0.wav")


if diffuse_audio == True:
  #####################
  #                   #
  #START JJ DIFFUSTION#
  #                   #
  #####################

  #######
  #START MAKE AUDIO CHUNKS FROM ITEM_0
  #######

  !pip install pydub
  from pydub import AudioSegment 
  from pydub.utils import make_chunks
  import os

  projectdirectory = save_directory

  workingpath = projectdirectory+"/level_0/"

  if not os.path.exists(workingpath+"item_0_chunks/"):
    os.makedirs(workingpath+"item_0_chunks/")
  else:
    for filename in os.listdir(workingpath+"item_0_chunks/"):
      os.remove(workingpath+"item_0_chunks/"+filename)


  myaudio = AudioSegment.from_file(workingpath+"item_0.wav", "wav") 
  chunk_length_ms = ((65536 / 1) / 44100) *1000 * 1 
  chunks = make_chunks(myaudio,chunk_length_ms)
  for i, chunk in enumerate(chunks): 
      chunk_name = workingpath+"item_0_chunks/{:08}.wav".format(i) 
      print ("exporting", chunk_name) 
      chunk.export(chunk_name, format="wav")

  #######
  #END MAKE AUDIO CHUNKS FROM ITEM_0
  #######

  #######
  #START UNLOAD JUKEBOX
  #######

  if True:
    del upsamplers
    empty_cache()
    upsamplers=None

  #######
  #END UNLOAD JUKEBOX
  #######



  #######
  #START DIFFUSION
  #######

  import subprocess

  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')

  !git clone https://github.com/reedmayhew18/sample-generator
  !git clone --recursive https://github.com/reedmayhew18/v-diffusion-pytorch
  !pip install /content/sample-generator
  !pip install /content/v-diffusion-pytorch

  import subprocess, os, sys, ipykernel

  def gitclone(url, targetdir=None):
      if targetdir:
          res = subprocess.run(['git', 'clone', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
      else:
          res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(res)

  def pipi(modulestr):
      res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(res)

  def pipie(modulestr):
      res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
      print(res)

  def wget(url, outputdir):
      # Using the !wget command instead of the subprocess to get the loading bar
      !wget $url -O $outputdir
      # res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
      # print(res)

  try:
      from google.colab import drive
      print("Google Colab detected. Using Google Drive.")
      is_colab = True
      google_drive = True
      save_models_to_google_drive = save_and_load_models_from_drive
  except:
      is_colab = False
      google_drive = False
      save_models_to_google_drive = False
      print("Google Colab not detected.")

  if is_colab:
      if google_drive is True:
          drive.mount('/content/gdrive')
          ai_root = '/content/gdrive/MyDrive/AI'
          root_path = f'{ai_root}/JuicyJukebox_Diffusion'
      else:
          root_path = '/content'
  else:
      root_path = os.getcwd()

  import os
  def createPath(filepath):
      os.makedirs(filepath, exist_ok=True)

  initDirPath = f'{root_path}/init_audio'
  createPath(initDirPath)
  outDirPath = f'{root_path}/audio_out'
  createPath(outDirPath)

  if is_colab:
      if google_drive and not save_models_to_google_drive or not google_drive:
          model_path = '/content/models'
          createPath(model_path)
      if google_drive and save_models_to_google_drive:
          model_path = f'{ai_root}/models'
          createPath(model_path)
  else:
      model_path = f'{root_path}/models'
      createPath(model_path)

  # libraries = f'{root_path}/libraries'
  # createPath(libraries)

  from prefigure.prefigure import get_all_args
  from contextlib import contextmanager
  from copy import deepcopy
  import math
  from pathlib import Path

  import sys

  from diffusion import sampling
  import torch
  from torch import optim, nn
  from torch.nn import functional as F
  from torch.utils import data
  from tqdm import trange
  from einops import rearrange

  import torchaudio
  from audio_diffusion.models import DiffusionAttnUnet1D
  import numpy as np
  import pydub
  from pydub import AudioSegment

  class DiffusionUncond(nn.Module):
      def __init__(self, global_args):
          super().__init__()

          self.diffusion = DiffusionAttnUnet1D(global_args, n_attn_layers = 4)
          self.diffusion_ema = deepcopy(self.diffusion)
          self.rng = torch.quasirandom.SobolEngine(1, scramble=True)

  import matplotlib.pyplot as plt
  import IPython.display as ipd


  def plot_and_hear(audio):
      haiaudio = ipd.Audio(audio.cpu().clamp(-1, 1), rate=44100)
      haiaudio = AudioSegment(haiaudio.data, frame_rate=44100, sample_width=2, channels=2)
      haiaudio.export(out_f=filename, format="wav")

  # --------
  # ATTENTION
  # --------
  # THIS IS THE FUNCTION THAT REPLACES PLOT_AND_HEAR AND SAVES THE CHUNKS OF PROCESSED AUDIO
  def save_jjd_audio(audio,filename):
      haiaudio = ipd.Audio(audio.cpu().clamp(-1, 1), rate=44100)
      haiaudio = AudioSegment(haiaudio.data, frame_rate=44100, sample_width=2, channels=2)
      haiaudio.export(out_f=filename, format="wav")

      #Can this be removed without crashing? Try it!
      #plt.plot(audio.cpu().t().numpy())
      
  def get_alphas_sigmas(t):
      """Returns the scaling factors for the clean image (alpha) and for the
      noise (sigma), given a timestep."""
      return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)

  def get_crash_schedule(t):
      sigma = torch.sin(t * math.pi / 2) ** 2
      alpha = (1 - sigma ** 2) ** 0.5
      return alpha_sigma_to_t(alpha, sigma)

  def t_to_alpha_sigma(t):
      """Returns the scaling factors for the clean image and for the noise, given
      a timestep."""
      return torch.cos(t * math.pi / 2), torch.sin(t * math.pi / 2)

  def alpha_sigma_to_t(alpha, sigma):
      """Returns a timestep, given the scaling factors for the clean image and for
      the noise."""
      return torch.atan2(sigma, alpha) / math.pi * 2


  sample_size = int(65536 * 1)
  sample_rate = 44100   
  latent_dim = 0             

  class Object(object):
      pass

  args = Object()
  args.sample_size = sample_size
  args.sample_rate = sample_rate
  args.latent_dim = latent_dim

  from urllib.parse import urlparse
  import hashlib
  model_name = "reed-and-sam-media_22pop"
  custom_ckpt_path = ''

  models_map = {
      "reed-and-sam-media_22pop": {'downloaded': False, 'sha': "4cd36b7071110c339649a118ca3ce9c5ac6d70e263f21ea9870d239eff5cb5e4", 'uri_list': ["https://reedandsammedia-2022pop-diffusionmodel.rpsmio.workers.dev/harmonai_22pop_200000.ckpt"] },
  }

  check_model_SHA = False

  def get_model_filename(diffusion_model_name):
      model_uri = models_map[diffusion_model_name]['uri_list'][0]
      model_filename = os.path.basename(urlparse(model_uri).path)
      return model_filename

  def download_model(diffusion_model_name, uri_index=0):
      if diffusion_model_name != 'custom':
          model_filename = get_model_filename(diffusion_model_name)
          model_local_path = os.path.join(model_path, model_filename)
          if os.path.exists(model_local_path) and check_model_SHA:
              print(f'Checking {diffusion_model_name} File')
              with open(model_local_path, "rb") as f:
                  bytes = f.read() 
                  hash = hashlib.sha256(bytes).hexdigest()
                  print(f'SHA: {hash}')
              if hash == models_map[diffusion_model_name]['sha']:
                  print(f'{diffusion_model_name} SHA matches')
                  models_map[diffusion_model_name]['downloaded'] = True
              else:
                  print(f"{diffusion_model_name} SHA doesn't match. Will redownload it.")
          elif os.path.exists(model_local_path) and not check_model_SHA or models_map[diffusion_model_name]['downloaded']:
              print(f'{diffusion_model_name} already downloaded. If the file is corrupt, enable check_model_SHA.')
              models_map[diffusion_model_name]['downloaded'] = True

          if not models_map[diffusion_model_name]['downloaded']:
              for model_uri in models_map[diffusion_model_name]['uri_list']:
                  wget(model_uri, model_local_path)
                  with open(model_local_path, "rb") as f:
                    bytes = f.read() 
                    hash = hashlib.sha256(bytes).hexdigest()
                    print(f'SHA: {hash}')
                  if os.path.exists(model_local_path):
                      models_map[diffusion_model_name]['downloaded'] = True
                      return
                  else:
                      print(f'{diffusion_model_name} model download from {model_uri} failed. Will try any fallback uri.')
              print(f'{diffusion_model_name} download failed.')

  if model_name == "custom":
    ckpt_path = custom_ckpt_path
  else:
    download_model(model_name)
    ckpt_path = f'{model_path}/{get_model_filename(model_name)}'

  model = DiffusionUncond(args)
  model.load_state_dict(torch.load(ckpt_path)["state_dict"])
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = model.requires_grad_(False).to(device)

  # # Remove non-EMA
  del model.diffusion

  import random
  import matplotlib.pyplot as plt
  import IPython.display as ipd
  from audio_diffusion.utils import Stereo, PadCrop
  from glob import glob

  model_fn = model.diffusion_ema

  def load_to_device(path, sr):
      audio, file_sr = torchaudio.load(path)
      if sr != file_sr:
        audio = torchaudio.transforms.Resample(file_sr, sr)(audio)
      audio = audio.to(device)
      return audio


  if diffusion_quality == "Super-Fast (Low-Quality)":
    steps = int(214 * 0.5)
    etamultiplier = 0.5
  elif diffusion_quality == "Fast":
    steps = int(214 * 1)
    etamultiplier = 1.0
  elif diffusion_quality == "Normal":
    steps = int(214 * 1.5)
    etamultiplier = 1.5
  elif diffusion_quality == "High-Quality":
    steps = int(214 * 2)
    etamultiplier = 2.0
  elif diffusion_quality == "Extreme":
    steps = int(214 * 3)
    etamultiplier = 3.0
  elif diffusion_quality == "Insane":
    steps = int(214 * 6)
    etamultiplier = 6.0

  if diffusion_strength == "Soften":
    noise_level = 0.05
  elif diffusion_strength == "Very-Mild":
    noise_level = 0.30
  elif diffusion_strength == "Mild":
    noise_level = 0.35
  elif diffusion_strength == "Normal":
    noise_level = 0.40
  elif diffusion_strength == "Moderate":
    noise_level = 0.45
  elif diffusion_strength == "Strong":
    noise_level = 0.50
  elif diffusion_strength == "Extreme":
    noise_level = 0.55
  elif diffusion_strength == "Insane":
    noise_level = 0.60

  # --------
  # ATTENTION
  # --------
  # THIS IS THE FUNCTION THAT DOES THE PROCESSING
  def run_jjd(inputfile):
    file_path = inputfile
    filenames = glob(file_path, recursive=True)

    augs = torch.nn.Sequential(
      PadCrop(args.sample_size, randomize=True),
      Stereo()
    )

    audio_sample = load_to_device(file_path, args.sample_rate)

    audio_sample = augs(audio_sample).unsqueeze(0)

    print("Rendering "+inputfile)

    t = torch.linspace(0, 1, steps + 1, device=device)
    step_list = get_crash_schedule(t)
    step_list = step_list[step_list < noise_level]

    alpha, sigma = t_to_alpha_sigma(step_list[-1])
    noised = torch.randn([1, 2, args.sample_size], device='cuda')
    noised = audio_sample * alpha + noised * sigma

    generated = sampling.iplms_sample(model_fn, noised, step_list.flip(0)[:-1], {})

    print("Done")
    save_jjd_audio(rearrange(generated, 'b d n -> d (b n)'),file_path+"_jjd.wav")

  import os

  # --------
  # ATTENTION
  # --------
  # SPECIFY DIRECTORY OF CHUNKS HERE:
  directory = save_directory+'/level_0/item_0_chunks/'
  # --------

  print("")
  print("Diffusion Time! Your estimated render time is about: " + str(int(((sample_length_in_seconds / 60) * 24) * etamultiplier))+" Minutes.")
  print("")

  for filename in sorted(os.listdir(directory)):
    if not filename.endswith("_jjd.wav") and not filename.endswith("_jjd-final.wav") and not filename.endswith("_jjd-final2.wav") and not filename.endswith("item_0.wav"):
      f = os.path.join(directory, filename)
      if os.path.isfile(f):
        run_jjd(f)

  finalhaioutput = AudioSegment.empty()

  for filename in sorted((os.listdir(directory))):
    if filename.endswith("_jjd.wav"):
      f = os.path.join(directory, filename)
      if os.path.isfile(f):
        #finalhaioutput += AudioSegment.from_file(f, format="wav")
        finalhaioutput = finalhaioutput.append(AudioSegment.from_file(f, format="wav"), crossfade=0 if not len(finalhaioutput) else 10)

  finalhaioutput.export(save_directory+"/level_0/item_0_jjd-final.wav", format="wav")

  print("")
  print("JuicyJukebox Diffusion Complete! Look for your render at: "+save_directory+"/level_0/item_0_jjd-final.wav")
  print("")
  #######
  #END DIFFUSION
  #######

  ##################
  #                #
  #END JJ DIFFUSION#
  #                #
  ##################

# Settings:

**save_directory:** Specify a **NEW** folder for Jukebox to create to save your render in. If you are resuming a previously saved project, put the path for that directory.

**audio_file:** Specifies which song Jukebox will continue. Upload the file you want (can be WAV, FLAC, MP3, M4A, OPUS, etc.) to the root directory of your Google Drive and fill in its name above. **UP TO 23 SECONDS OF AUDIO WILL BE USED.** Any remaining audio will be trimmed.

**PLEASE NOTE:** Primer lengths will be rounded down to the nearest whole number. (Example: with a 16.4 second primer, only 16.0 seconds of audio will be used.)

**generation_length_in_seconds:** How much audio Jukebox will generate in ADDITION to the provided audio.

**rendering/upsampling quality:** How fast/high-quality Jukebox will sample new audio. Higher quality settings will allow Jukebox to "remember" more audio and take smaller steps while rendering. **Selecting BPM will dynamically adjust the conditioning based on the BPM of the song.**

**sampling_agressiveness:** Determines the creativity and energy of Jukebox. The higher the agressiveness, the more chaotic the result will be.

#Credits

Join the Jukebox community at https://discord.gg/aEqXFN9amV

Big thank you to MichaelsLab for his coding with Speed Upsampling, otherwise, this notebook would not be possible. 

Also, thank you to Broccaloo for the One-Click notebook which this notebook is based off of: https://www.youtube.com/c/broccaloo

Diffusion Model Trained by Reed Mayhew with Reed + Sam Media

Important links:

Official blog: https://openai.com/blog/jukebox/
Original repo: https://github.com/openai/jukebox/


# Licenses

Jukebox License: Non-commercial, for details see: https://github.com/openai/jukebox/blob/master/LICENSE



---


Diffusion Model License: 2022Pop Model protected by Reed + Sam Media and may not be redistributed or used without written permission. Audio output from model has no restrictions for use or copyright claims. Reed + Sam Media assumes no legal responsibility for the use of or audio outputted from the diffusion model. User is soley responsible for any audio inputted, processed, and outputted through the diffusion model. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


---


Diffusion Code License:
Original notebook by Zach Evans (https://github.com/zqevans, https://twitter.com/zqevans). Overall structure and setup code taken from Disco Diffusion (https://www.discodiffusion.com) Interpolation and audio display code from CRASH inference notebook (https://github.com/simonrouard/CRASH). Licensed under the MIT License. Copyright (c) 2022 Zach Evans. Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

#Change Log:

2022-09-30 at 09:45 AM EST:

1. Updated notebook settings to be compatible with Google Colab credits.

2. Added "Soften" diffusion option.

3. Changed default selections to be more optimal.

2022-08-30 at 12:00 AM EST:

1. Introducing: JuicyJukebox Diffusion! Post-process your Jukebox render through A.I. diffusion to add more detail, restore high-frequencies, and add stereo sound. (BETA)

2022-08-01 at 10:10 AM EST:

1. HUGE speed enhancements! Most noticable at higher rendering_quality settings. Enhancements reduce High-Quality & BPM render times very close to Fast render times by adjusting chunk sizes. (Thank you to Jimney Baltigor for the helpful tips with Jukebox code!)

2. Added support for A100 GPUs to take advantage of their larger VRAM capacity to decrease speed-upsampling render times.

2022-07-16 at 04:00 PM EST:

1. Fixed an error inside the Google Drive save/load model function where the URL specified to download 5B_L1 from Asure was directed towards the 5B_L0 model. (Thanks, Broccaloo!)

2022-07-08 at 07:00 AM EST:

1. Fixed a syntax error that caused bpm_fraction to be calculated when a BPM option was not selected.

2022-07-05 at 09:50 PM EST:

1. Locked round_primer_duration_down_to_nearest_integer to "True" to avoid possible glitching when jukebox transitions from primed audio to generated audio.

2. Fixed a syntax error that caused bpm_fraction variables not to be defined.

2022-07-05 at 12:58 PM EST:

1. Updated how the primer duration is calulated to fix a bug where upsampling may result in random noise with primers shorter than 23 seconds.

2. Temperature values set by sampling_aggressiveness now vary slightly between quality levels to more accurately reflect their descriptions. Larger hop fraction sizes result in Jukebox being "more creative" at the same temperature values, and vice versa. For level-2 sampling at Fast quality, temp is reduced by 0.002. For High-Quality and BPM, temp is increased by 0.002. For BPMx2, temp is increased by 0.004.

3. Added the option to round-down the duration of the primer to the nearest whole number to help reduce the amount of effort needed to prepare primer audio.
